In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import pickle
import sentence_transformers
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from catboost import CatBoostClassifier

In [ ]:
train_path = 'data/train.csv'
val_path = 'data/val.csv'
test_path = 'data/test.csv'
SEED = 42

In [ ]:
df_train = pd.read_csv(train_path)
df_train.head()

In [ ]:
df_val = pd.read_csv(val_path)
df_val.head()

In [ ]:
df_test = pd.read_csv(test_path)
df_test.head()

In [ ]:
X_train, y_train = df_train['text'], df_train['label']
X_val, y_val = df_val['text'], df_val['label']
X_test = df_test['text']

In [ ]:
del X_train, X_val, X_test, df_train, df_val, df_test

In [ ]:
# bert_model = sentence_transformers.SentenceTransformer('bert-base-nli-mean-tokens')

# train_encodings = bert_model.encode(X_train)
# print(len(train_encodings))
# val_encodings = bert_model.encode(X_val)
# print(len(val_encodings))
# test_encodings = bert_model.encode(X_test)
# print(len(test_encodings))

# train_encodings = np.array(train_encodings)
# val_encodings = np.array(val_encodings)
# test_encodings = np.array(test_encodings)

# with open('train_encodings.npy', 'wb') as f:
#     np.save(f, train_encodings)
# with open('val_encodings.npy', 'wb') as f:
#     np.save(f, val_encodings)
# with open('test_encodings.npy', 'wb') as f:
#     np.save(f, test_encodings)

In [ ]:
with open('train_encodings.npy', 'rb') as f:
    train_encodings = np.load(f)
with open('val_encodings.npy', 'rb') as f:
    val_encodings = np.load(f)
with open('test_encodings.npy', 'rb') as f:
    test_encodings = np.load(f)

In [ ]:
cbc = CatBoostClassifier(learning_rate=0.1582, iterations=10000)

In [ ]:
cbc.fit(train_encodings, y_train)

In [ ]:
y_pred = cbc.predict(val_encodings)

In [ ]:
precision = precision_score(y_val, y_pred, average='micro')
recall = recall_score(y_val, y_pred, average='micro')
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')

In [ ]:
print(accuracy)
print(precision)
print(recall)
print(f1)

In [ ]:
y_test = cbc.predict(test_encodings)

In [ ]:
submission = pd.DataFrame(y_test)
submission.to_csv('data/submission.csv', header=['label'], index=False)